В качестве лабораторной работы по теме "Ансамбли моделей" предлагается участие в [соревановании на Kaggle](https://www.kaggle.com/competitions/ensembles-competition/leaderboard).

Основные тезисы и правила



1.   В этом семестре обязательным требованием является размещение результатов выполнения лабораторных работ (всех, начиная с текущей) в своем репозитории на github. Формат — блокнот .ipynb, содержащий программный код и необходимые пояснения markdown.
2.   Топ-3 команды, которые рассказывали о своей работе, получают автоматически 5 по этой лабе (при выполнении пункта 1. можно общий блокнот, если он один на всех, но в своем репозитории с кратким описанием своего вклада в общую работу).
3. Остальные участники размещают также свои материалы у себя в репозитории. В случае командной работы тоже описать свой вклад (конкретно вы, что делали в рамках проекта).
4. Без защиты можно автоматически получить оценку 3. Для более высокой оценки необходимо защитить свою работу. Защита в виде беседы, где вы опять же расскажате о том, как добились результата и о своем вкладе в случае командной работы.





In [ ]:
# Подгрузка исходных данных
!gdown 1Jk3kcdU8VZwJsrcyTsbr9GLj4uSJhST4
!unzip ensembles-competition.zip

In [ ]:
!pip install -q catboost
!pip install -q lightgbm
!pip install -q xgboost
!pip install -q mlxtend
!pip install -q dask[dataframe]
!pip install optuna
!pip install LightGBM

In [ ]:
import pandas as pd
import json
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.decomposition import PCA

import numpy as np
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor

from sklearn.metrics import mean_absolute_error
import optuna

In [ ]:
X_train = pd.read_csv('train_contest.csv')
X_test = pd.read_csv('for_prediction.csv')
y_train = X_train['mean_salary']
X_train = X_train.drop(columns=['mean_salary'])

## Подготовка данных

In [ ]:
def dataframe_convert_bool_to_int(dataframe):
    bool_columns = dataframe.select_dtypes(include=['bool']).columns
    for col in bool_columns:
        dataframe[col] = dataframe[col].astype(int)

    return dataframe


def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    return text.strip()


def combine_dict_values(x, key):
    if pd.isna(x) or x == '[]':
        return None
    try:
        dict_list = ast.literal_eval(x)
        return ','.join([d[key] for d in dict_list])
    except:
        return None


def convert_to_lowercase(dataframe: pd.DataFrame) -> pd.DataFrame:
    text_columns = dataframe.select_dtypes(include=['object', 'category']).columns

    for column in text_columns:
        dataframe[column] = dataframe[column].apply(lambda x: x.lower() if isinstance(x, str) else x)

    return dataframe

In [ ]:
def add_vector_embeddings(dataframe: pd.DataFrame):
    text_columns = dataframe.select_dtypes(include=['object', 'category']).columns

    for column in text_columns[:]:
        print(column)
        dataframe[column+'_embedding'] = dataframe[column].apply(lambda x: model.encode(x))
    print('embeddings added')
    return dataframe

In [ ]:
def preprocess_data(salary):
    # Удаление не нужных признаков
    columns_to_drop_null = ["response_url", "sort_point_distance", "insider_interview", "relations", "working_days", "working_time_intervals", "working_time_modes", "department", 'address'] # address, department и area - вопрос как лучше с ними или без
    columns_to_drop_unused = ["alternate_url", "url", "immediate_redirect_url", "contacts", "published_at", "created_at"]
    salary = salary.drop(columns=columns_to_drop_null+columns_to_drop_unused)
    salary = dataframe_convert_bool_to_int(salary)

    # Достаем из json формата данные
    salary['area'] = salary['area'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )
    salary['type'] = salary['type'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )

    salary['employer'] = salary['employer'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['name']
    )
    salary['schedule'] = salary['schedule'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['id']
    )
    salary['experience'] = salary['experience'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['name']
    )

    salary['key_skills'] = salary['key_skills'].apply(combine_dict_values, key='name')
    salary['specialization_name'] = salary['specializations'].apply(combine_dict_values, key='name')
    salary['specialization_profarea_name'] = salary['specializations'].apply(combine_dict_values, key='profarea_name')
    salary['snippet'] = salary['snippet'].apply(
        lambda x: None if pd.isna(x) else ast.literal_eval(x)['requirement']
    )
    salary.drop(columns=['specializations'], inplace=True)

    salary["description"] = salary["description"].apply(preprocess_text)
    salary.fillna('empty', inplace=True)

    salary = convert_to_lowercase(salary)
    if labels_to_drop is not None:
        salary = salary.drop(columns=labels_to_drop)
        salary = pd.get_dummies(salary)
    return salary

## Optuna

In [ ]:
X_train_prep = preprocess_data(X_train)

In [ ]:
X = np.array(X_train_prep)
y = y_train.copy()

In [ ]:
def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    param = {
        'metric': 'mae',
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10.0,log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10.0,log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2,log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 200),
        'num_leaves': trial.suggest_int('num_leaves', 40, 500),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 500),
        'cat_smooth': trial.suggest_int('cat_smooth', 1, 300),
        'force_col_wise':True
    }

    model = LGBMRegressor(**param, n_jobs=-1, verbosity=-1)

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    return mae

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

In [ ]:
param = study.best_params
model = LGBMRegressor(**param)
model.fit(np.array(X), y)

y_pred_optuna = model.predict(np.array(X_test_prep))

## Подготовка предсказания

In [ ]:
%%capture
!warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

X_test_prep = preprocess_data(X_test)

missing_cols = set(X_train_prep.columns) - set(X_test_prep.columns)
for col in missing_cols:
    X_test_prep[col] = 0

X_test_prep = X_test_prep[X_train_prep.columns]

In [ ]:
def pred_to_csv(y_pred, name_file):
    df_result = pd.DataFrame(data={"Predicted":list(y_pred)})
    df_result = df_result.reset_index()
    df_result = df_result.rename(columns={"index":"Id"})
    df_result.to_csv(f"{name_file}.csv", index=False)

In [ ]:
pred_to_csv(y_pred_optuna, "submit.csv")

## Выводы:

1. **Наше решение:** предобработка признаков, признаки в эмбединги + обучение модели LGBMRegressor с подбором параметров с помощью Optuna
2. **Наша команда:** Андрей, Женя, Гоша, Марат. Андрей, Женя - подготовка, отбор и очистка фич, Марат - формирование эмбедингов, тестирование альтернативных подходов, Гоша - тестирование и сравнение различных моделей + настройка Optuna.
3. **В итоге:** 4 место, немного не хватило до 3.